In [ ]:
#!pip install langchain

In [ ]:
from langchain.agents import AgentExecutor
from langchain.pydantic_v1 import BaseModel

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

In [ ]:
#!pip install openai

In [ ]:
#!pip install google-api-python-client google-auth-oauthlib google-auth-httplib2 beautifulsoup4

In [ ]:
from langchain import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent
from langchain.sql_database import SQLDatabase
from langchain.agents.agent_types import AgentType
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.agents.agent_toolkits import GmailToolkit

In [ ]:
db = SQLDatabase.from_uri("sqlite:///resources/users.db")
db_toolkit = SQLDatabaseToolkit(db=db, llm=OpenAI(temperature=0))

In [ ]:
gmail_toolkit = GmailToolkit()

# Agente SQL + GMAIL

In [ ]:
llm=ChatOpenAI(temperature=0,model_name="gpt-4")

In [ ]:
agent = initialize_agent(
    llm=llm,
    tools=gmail_toolkit.get_tools()+db_toolkit.get_tools(),
    verbose=True,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
)

In [ ]:
agent

In [ ]:
agent.run('Busca los 2 usuarios que más han comprado y envía el siguiente correo:'
          '\nAsunto: Código de descuento por compras'
          '\nContenido: "Hola, gracias por tus compras. Tienes un bono de descuento en tu siguiente compra.' 
          '\nUtiliza el código: #LLMConEsteroides y obtén un 50% de descuento."')

### Other


In [ ]:
agent = initialize_agent(
    llm=llm,
    tools=gmail_toolkit.get_tools()+db_toolkit.get_tools(),
    verbose=True,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    return_intermediate_steps =True
)


In [ ]:
agent_executor = AgentExecutor(
    agent=agent, 
    max_iterations=5,
    tools=gmail_toolkit.get_tools()+db_toolkit.get_tools(),
    return_intermediate_steps=True
) | (lambda x: x["output"])


In [ ]:

class AgentInputs(BaseModel):
    query: str


In [ ]:
inp  = AgentInputs(query='Busca los 2 usuarios que más han comprado y envía el siguiente correo:'
          '\nAsunto: Código de descuento por compras'
          '\nContenido: "Hola, gracias por tus compras. Tienes un bono de descuento en tu siguiente compra.' 
          '\nUtiliza el código: #LLMConEsteroides y obtén un 50% de descuento."')

In [ ]:
#agent_executor = agent_executor.with_types()

In [ ]:
agent_executor.invoke({"input":'Busca los 2 usuarios que más han comprado y envía el siguiente correo:'
          '\nAsunto: Código de descuento por compras'
          '\nContenido: "Hola, gracias por tus compras. Tienes un bono de descuento en tu siguiente compra.' 
          '\nUtiliza el código: #LLMConEsteroides y obtén un 50% de descuento."'})